In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
from finta import TA

In [4]:
fut1 = yf.Ticker("GEZ22.CME")

fut1_df = fut1.history(period="150d", interval="1h")
fut1_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2022-04-25 00:00:00-04:00,96.870003,96.879997,96.855003,96.855003,0,0,0
2022-04-25 01:00:00-04:00,96.849998,96.849998,96.834999,96.845001,962,0,0
2022-04-25 02:00:00-04:00,96.839996,96.894997,96.839996,96.889999,7839,0,0
2022-04-25 03:00:00-04:00,96.889999,96.904999,96.875000,96.879997,8918,0,0
2022-04-25 04:00:00-04:00,96.875000,96.919998,96.860001,96.889999,7921,0,0
...,...,...,...,...,...,...,...
2022-10-14 14:00:00-04:00,94.900002,94.904999,94.894997,94.900002,14045,0,0
2022-10-14 15:00:00-04:00,94.900002,94.904999,94.889999,94.894997,7201,0,0
2022-10-14 16:00:00-04:00,94.889999,94.894997,94.885002,94.894997,10050,0,0
2022-10-16 18:00:00-04:00,94.889999,94.910004,94.875000,94.904999,326,0,0


In [5]:
fut1_df = fut1_df.drop(columns=["Dividends", "Stock Splits"])

fut1_df

,Open,High,Low,Close,Volume
2022-04-25 00:00:00-04:00,96.870003,96.879997,96.855003,96.855003,0
2022-04-25 01:00:00-04:00,96.849998,96.849998,96.834999,96.845001,962
2022-04-25 02:00:00-04:00,96.839996,96.894997,96.839996,96.889999,7839
2022-04-25 03:00:00-04:00,96.889999,96.904999,96.875000,96.879997,8918
2022-04-25 04:00:00-04:00,96.875000,96.919998,96.860001,96.889999,7921
...,...,...,...,...,...
2022-10-14 14:00:00-04:00,94.900002,94.904999,94.894997,94.900002,14045
2022-10-14 15:00:00-04:00,94.900002,94.904999,94.889999,94.894997,7201
2022-10-14 16:00:00-04:00,94.889999,94.894997,94.885002,94.894997,10050
2022-10-16 18:00:00-04:00,94.889999,94.910004,94.875000,94.904999,326


In [6]:
fut2 = yf.Ticker("CLZ22.NYM")

fut2_df = fut2.history(period="150d", interval="1h")
fut2_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2022-04-25 00:00:00-04:00,90.790001,90.980003,90.599998,90.650002,0,0,0
2022-04-25 01:00:00-04:00,90.739998,91.120003,90.430000,90.629997,300,0,0
2022-04-25 02:00:00-04:00,90.629997,90.739998,90.050003,90.260002,1415,0,0
2022-04-25 03:00:00-04:00,90.209999,90.629997,90.089996,90.190002,590,0,0
2022-04-25 04:00:00-04:00,90.180000,90.250000,89.690002,89.709999,1554,0,0
...,...,...,...,...,...,...,...
2022-10-14 15:00:00-04:00,84.760002,84.889999,84.570000,84.779999,3608,0,0
2022-10-14 16:00:00-04:00,84.769997,84.820000,84.540001,84.540001,2527,0,0
2022-10-16 18:00:00-04:00,84.629997,85.260002,84.500000,85.230003,2608,0,0
2022-10-16 19:00:00-04:00,85.250000,85.260002,85.180000,85.209999,52,0,0


In [7]:
fut2_df = fut2_df.drop(columns=["Dividends", "Stock Splits"])

fut2_df

,Open,High,Low,Close,Volume
2022-04-25 00:00:00-04:00,90.790001,90.980003,90.599998,90.650002,0
2022-04-25 01:00:00-04:00,90.739998,91.120003,90.430000,90.629997,300
2022-04-25 02:00:00-04:00,90.629997,90.739998,90.050003,90.260002,1415
2022-04-25 03:00:00-04:00,90.209999,90.629997,90.089996,90.190002,590
2022-04-25 04:00:00-04:00,90.180000,90.250000,89.690002,89.709999,1554
...,...,...,...,...,...
2022-10-14 15:00:00-04:00,84.760002,84.889999,84.570000,84.779999,3608
2022-10-14 16:00:00-04:00,84.769997,84.820000,84.540001,84.540001,2527
2022-10-16 18:00:00-04:00,84.629997,85.260002,84.500000,85.230003,2608
2022-10-16 19:00:00-04:00,85.250000,85.260002,85.180000,85.209999,52


In [70]:
# Visualize close price for the investment
security_close = btc_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_upper = btc_df[["BB_UPPER"]].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


bb_middle = btc_df[["BB_MIDDLE"]].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = btc_df[["BB_LOWER"]].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)


# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower
bbands_plot

:Overlay
   .Curve.I   :Curve   [index]   (Close)
   .Curve.II  :Curve   [index]   (BB_UPPER)
   .Curve.III :Curve   [index]   (BB_MIDDLE)
   .Curve.IV  :Curve   [index]   (BB_LOWER)

In [71]:
rsi_df = TA.RSI(btc_df)

rsi_df

2022-03-14 09:30:00-04:00          NaN
2022-03-14 10:30:00-04:00     0.000000
2022-03-14 11:30:00-04:00     0.000000
2022-03-14 12:30:00-04:00     0.000000
2022-03-14 13:30:00-04:00     0.000000
                               ...    
2022-10-14 12:30:00-04:00    45.743005
2022-10-14 13:30:00-04:00    43.828385
2022-10-14 14:30:00-04:00    44.615167
2022-10-14 15:30:00-04:00    42.570149
2022-10-14 16:00:00-04:00    42.489071
Name: 14 period RSI, Length: 1051, dtype: float64

In [72]:
#btc_df = pd.concat([btc_df, rsi_df], axis=1)
btc_df["RSI"] = rsi_df

btc_df

,Open,High,Low,Close,Volume,BB_UPPER,BB_MIDDLE,BB_LOWER,RSI
2022-03-14 09:30:00-04:00,420.890015,423.480011,419.109985,422.790009,20055923,NaN,NaN,NaN,NaN
2022-03-14 10:30:00-04:00,422.790009,424.549988,422.059998,422.269989,11182488,NaN,NaN,NaN,0.000000
2022-03-14 11:30:00-04:00,422.279999,422.459991,418.839996,419.809998,10523034,NaN,NaN,NaN,0.000000
2022-03-14 12:30:00-04:00,419.804993,420.170013,417.299988,417.420013,10709894,NaN,NaN,NaN,0.000000
2022-03-14 13:30:00-04:00,417.420105,417.649994,415.790009,415.899994,10163349,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...
2022-10-14 12:30:00-04:00,359.839996,360.790009,358.510010,359.411194,9799514,367.000895,359.969571,352.938246,45.743005
2022-10-14 13:30:00-04:00,359.411011,360.170013,357.799988,358.380005,9034646,366.978452,360.032571,353.086690,43.828385
2022-10-14 14:30:00-04:00,358.364990,359.230011,357.430115,358.704987,12121847,366.972629,360.081320,353.190011,44.615167
2022-10-14 15:30:00-04:00,358.704987,359.429993,356.959991,357.670013,20260153,366.981210,360.044821,353.108433,42.570149


In [73]:
btc_signals_df = pd.DataFrame()
btc_signals_df["Close"] = btc_df["Close"]

In [74]:
# Create a trading algorithm using Bollinger Bands
# Set the Signal column
btc_signals_df["Signal"] = 0.0

# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the Close price is less than the BB_LOWER window
# where -1 is when the Close price is greater the the BB_UPPER window
for index, row in btc_df.iterrows():
    if (row["Close"] < row["BB_LOWER"]) and ((row["RSI"] <= 30) or (row["RSI"] > 50 and row["RSI"] < 70)):
        btc_signals_df.loc[index, "Signal"] = 1.0
    if (row["Close"] > row["BB_UPPER"]) and ((row["RSI"] >= 70) or (row["RSI"] > 30 and row["RSI"] < 50)):
        btc_signals_df.loc[index,"Signal"] = -1.0

btc_signals_df["Signal"] = btc_signals_df["Signal"].astype(int)
btc_signals_df["Entry/Exit"] = btc_signals_df["Signal"].diff()        
        
# Review the DataFrame
btc_signals_df

,Close,Signal,Entry/Exit
2022-03-14 09:30:00-04:00,422.790009,0,NaN
2022-03-14 10:30:00-04:00,422.269989,0,0.0
2022-03-14 11:30:00-04:00,419.809998,0,0.0
2022-03-14 12:30:00-04:00,417.420013,0,0.0
2022-03-14 13:30:00-04:00,415.899994,0,0.0
...,...,...,...
2022-10-14 12:30:00-04:00,359.411194,0,0.0
2022-10-14 13:30:00-04:00,358.380005,0,0.0
2022-10-14 14:30:00-04:00,358.704987,0,0.0
2022-10-14 15:30:00-04:00,357.670013,0,0.0


In [75]:
# Visualize entry position relative to close price
entry = btc_signals_df[btc_signals_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = btc_signals_df[btc_signals_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = btc_signals_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_upper = btc_df[["BB_UPPER"]].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


bb_middle = btc_df[["BB_MIDDLE"]].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = btc_df[["BB_LOWER"]].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)


# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower * entry * exit
bbands_plot

:Overlay
   .Curve.I          :Curve   [index]   (Close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (Close)
   .Scatter.Close.II :Scatter   [index]   (Close)

In [76]:
z = 1
PL = 0.00
start_price = (btc_df['Close'].head(1))
start_price = float(start_price)

end_price = (btc_df['Close'].tail(1))
end_price = float(end_price)

In [77]:
print("$BTC BBANDS/RSI Trading Results:", "\n")
print("The starting price is: $", round(start_price,2))
print("The ending price is: $", round(end_price,2), "\n")

#Getting buy and sell dates
for index, row in btc_signals_df.iterrows():
    if row['Signal'] == 1:
        if row['Entry/Exit'] != -1:
            if z == 1:
                print (index, round(row['Close'],2), '- Buy')
                close = row['Close']
                PL = PL - close
                z = z - 1 
    else:
        if row['Entry/Exit'] == -1:
            if z == 0:
                print (index, round(row['Close'],2), '- Sell')
                close = row['Close'] 
                PL = PL + close
                returns = (PL / start_price)
                return_pct = "{:.2%}".format(returns)
                print ("Total Profit/Loss is $",round(PL,2))
                print ("Total Return Percentage is", return_pct, "\n")
                z = z + 1
                    

                    
entire_hold_return = (end_price - start_price)
entire_hold_pct = "{:.2%}".format((end_price - start_price) / start_price)
print("The return for holding for the entire period was $", round(entire_hold_return,2), "or", entire_hold_pct, ".")
PL = 0.00   

$BTC BBANDS/RSI Trading Results: 

The starting price is: $ 422.79
The ending price is: $ 357.63 

2022-04-06 09:30:00-04:00 444.98 - Buy
2022-04-06 12:30:00-04:00 445.82 - Sell
Total Profit/Loss is $ 0.84
Total Return Percentage is 0.20% 

2022-04-22 09:30:00-04:00 432.94 - Buy
2022-04-22 13:30:00-04:00 428.9 - Sell
Total Profit/Loss is $ -3.2
Total Return Percentage is -0.76% 

2022-05-12 13:30:00-04:00 386.96 - Buy
2022-05-12 14:30:00-04:00 388.17 - Sell
Total Profit/Loss is $ -1.98
Total Return Percentage is -0.47% 

2022-06-09 15:30:00-04:00 401.45 - Buy
2022-06-10 12:30:00-04:00 391.52 - Sell
Total Profit/Loss is $ -11.92
Total Return Percentage is -2.82% 

2022-06-13 09:30:00-04:00 377.18 - Buy
2022-06-13 10:30:00-04:00 377.39 - Sell
Total Profit/Loss is $ -11.71
Total Return Percentage is -2.77% 

2022-06-30 09:30:00-04:00 373.53 - Buy
2022-06-30 10:30:00-04:00 377.26 - Sell
Total Profit/Loss is $ -7.98
Total Return Percentage is -1.89% 

2022-07-14 09:30:00-04:00 371.97 - Buy
